In [1]:
from ultralytics import YOLO
import json
from yolo_config import Config
import os

In [26]:
for model_size in Config.MODEL_SIZES:
    for threshold in Config.THRESHOLDS:
        for fold in range(1,11):

            # Weights From Training

            threshold_dir = f'{model_size}_conf={threshold}'
            fold_dir = f'fold_{fold}'

            yaml_path = os.path.join(Config.DATASET, fold_dir, 'data.yaml')

            best_weights_path = os.path.join(Config.RUNS_DIR, Config.DATASET, model_size,
                                             fold_dir, 'train', 'weights','best.pt')

            model = YOLO(best_weights_path)

            # Validate for every threshold

            validation_project = os.path.join(Config.RUNS_DIR, Config.DATASET, 
                                              threshold_dir, fold_dir)
            metrics = model.val(
                data = yaml_path, 
                split = 'val', 
                conf = threshold, 
                project = validation_project, 
                save_json = True
            )
            
            tp, fp, fn, tn =  metrics.confusion_matrix.matrix.flatten()

            results = metrics.results_dict
            results.update(
                {
                'TP': tp,
                'FP': fp,
                'FN': fn,
                'TN': tn,
            }
            )

            results_path = os.path.join(validation_project, 'val', 'results.json')


            with open(results_path, 'w') as file:
                json.dump(results, file)

            # break # Fold
        # break # Confidence
    break # Model Size


Ultralytics YOLOv8.0.164  Python-3.8.17 torch-2.0.1+cpu CPU (11th Gen Intel Core(TM) i7-1165G7 2.80GHz)
Model summary (fused): 168 layers, 3005843 parameters, 0 gradients
val: Scanning C:\Users\Alejandro Duque\Documents\USFQ\Proyecto Integrador\Workspace\Yolo\InBreast_Yolov8\fold_1\valid\labels.cache... 19 images, 11 backgrounds, 0 corrupt: 100%|██████████| 19/19 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 2/2 [00:03<00:00,  1.72s/it]
                   all         19          8      0.886      0.978      0.982      0.799
Speed: 2.1ms preprocess, 159.2ms inference, 0.0ms loss, 0.5ms postprocess per image
Saving runs_final\InBreast_Yolov8\yolov8n.pt_conf_0.1\fold_1\val\predictions.json...
Results saved to runs_final\InBreast_Yolov8\yolov8n.pt_conf_0.1\fold_1\val
Ultralytics YOLOv8.0.164  Python-3.8.17 torch-2.0.1+cpu CPU (11th Gen Intel Core(TM) i7-1165G7 2.80GHz)
Model summary (fused): 168 layers, 3005843 p